# BoneawareAI

By: Karthik Subramanian, Charles Green, Sai Anurag Pichika, Saarang Prabhuram


## Setup

### Load Extensions

Before getting started we need to run some standard code to set up our environment. You'll need to execute this code again each time you start the notebook.

First, run this cell to load the [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload) extension. This enables us to modify `.py` source files and reintegrate them into the notebook, ensuring a smooth editing and debugging experience.

In [1]:
%load_ext autoreload
%autoreload 2

### Google Colab Setup
Next we need to run a few commands to set up our environment on Google Colab. If you are running this notebook on a local machine you can skip this section.

Run the following cell to mount your Google Drive. Follow the link, sign in to your Google account (the same account you used to store this notebook!).

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
import os
PROJECT_PATH = 'BoneawareAI'
GOOGLE_DRIVE_PATH = f'/content/drive/MyDrive/{PROJECT_PATH}'
os.chdir(GOOGLE_DRIVE_PATH)
os.getcwd()

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/BoneawareAI'

In [4]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH) # this is important for the imports in the .py files to work

In [5]:
!pip install pyyaml==5.4.1
!pip install boto3
!pip install configparser
!pip install torch

### Local Setup OR Google Drive
Run the cell below regardless of whether you are using google drive or local setup.

In [6]:
# if running locally set GOOGLE PATH
import sys
if 'google.colab' in sys.modules:
  print(f'Running in google colab. Our path is `{GOOGLE_DRIVE_PATH}`')
else:
  GOOGLE_DRIVE_PATH = '.'
  print('Running locally.')

Running locally.


### Imports

In [7]:
import sys
sys.path.append('../src')  # Add the 'src' folder to Python's module search path
sys.path.append('../datasets')  # Add the 'datasets' folder to Python's module search path
sys.path.append('../notebooks')  # Add the 'notebooks' folder to Python's module search path

## Data Preprocessing
Get the dataset, perform data augmentation to get finalized MURA dataset

In [16]:
# Downloading MURA dataset and unzipping the file (this one takes time)
from src.data_loader import download_dataset
from src.constants import DATASETS_FOLDER, MURA_DATASET
from src.helpers.utils import unzip_file
download_dataset(MURA_DATASET, DATASETS_FOLDER)
unzip_file(os.path.join(os.getcwd(), DATASETS_FOLDER, MURA_DATASET))

File downloaded successfully to datasets\MURA-v1.1.zip
successfully unzipped the file at path c:\code\BoneawareAI\datasets\MURA-v1.1.zip


In [8]:
from image_utils import MURADataset, get_transforms, load_data

In [21]:
# Set paths
data_dir = "../datasets/MURA-v1.1"
batch_size = 32

# Load Data
train_loader, valid_loader = load_data(data_dir, batch_size=batch_size)

# Test the DataLoader
for images, labels in train_loader:
    print(f"Batch size: {len(images)}, Labels: {labels}")
    break

Found 36808 validated image filenames belonging to 1 classes in the training set.
Found 3197 validated image filenames belonging to 1 classes in the validation set.
Batch size: 32, Labels: tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])


In [19]:
data_dir = "../datasets/MURA-v1.1"
batch_size = 32

# Load training and validation data
train_loader, valid_loader = load_data(data_dir, batch_size=batch_size)

# Test the training DataLoader
print("Training Data:")
for images, labels in train_loader:
    print(f"Batch size: {len(images)}, Labels: {labels}")
    break

# Test the validation DataLoader
print("Validation Data:")
for images, labels in valid_loader:
    print(f"Batch size: {len(images)}, Labels: {labels}")
    break

Training Data:
Raw img_path from CSV: MURA-v1.1/train/XR_ELBOW/patient05171/study1_positive/image2.png
Adjusted img_path: XR_ELBOW/patient05171/study1_positive/image2.png
Constructed full_img_path: ..\datasets\MURA-v1.1\train\XR_ELBOW\patient05171\study1_positive\image2.png
Raw img_path from CSV: MURA-v1.1/train/XR_HAND/patient06561/study1_positive/image1.png
Adjusted img_path: XR_HAND/patient06561/study1_positive/image1.png
Constructed full_img_path: ..\datasets\MURA-v1.1\train\XR_HAND\patient06561\study1_positive\image1.png
Raw img_path from CSV: MURA-v1.1/train/XR_WRIST/patient06448/study1_positive/image1.png
Adjusted img_path: XR_WRIST/patient06448/study1_positive/image1.png
Constructed full_img_path: ..\datasets\MURA-v1.1\train\XR_WRIST\patient06448\study1_positive\image1.png
Raw img_path from CSV: MURA-v1.1/train/XR_ELBOW/patient05395/study1_positive/image1.png
Adjusted img_path: XR_ELBOW/patient05395/study1_positive/image1.png
Constructed full_img_path: ..\datasets\MURA-v1.1\tra